### First Regression (Turnover intention ~ unfair treatment x neg. reciprocity)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

### 1.  Read in SOEP Data:
- vp : 2005 data : main variables of interest: questions on negative reciprocity
- wp: 2006 data : main variables of interest: question on perceived recognition for work
- xp: 2007 data : main variables of interest: turnover intentions, controls

In [ ]:
# define path: insert the path where the SOEP data is stored on your computer here
from pathlib import Path
## copy your path in here:
# Path Max: C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/SOEP-Data/Stata/raw
# Path Maxie: /Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw

data_folder = Path("C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/SOEP-Data/Stata/raw")
# define relevant subsets of SOEP-data
file_names = ['vp', 'wp', 'xp']

file_paths = [data_folder / f"{file_name}.dta" for file_name in file_names]
# some controls are in gen data
file_paths_2 = [data_folder / f"{file_name}gen.dta" for file_name in file_names]

In [262]:
# read in 2005 data for the reciprocity measures
data05 = pd.read_stata(file_paths[0], columns=["pid","hid", "syear","vp12602", "vp12603", "vp12605"]).set_index(['pid', 'hid'])
df_2005 = data05.rename(columns={ 'vp12602': 'take_revenge', 'vp12603': 'similar_problems', 'vp12605': 'insult_back'})
# create dummies for take_revenge question
# Create dummy variables
dummies = pd.get_dummies(df_2005['similar_problems'])

# Join the dummy variables to the original dataframe
df_2005 = pd.concat([df_2005, dummies], axis=1)
# rename dummy
df_05 = df_2005.rename(columns = {'[1] Trifft ueberhaupt nicht zu' : 'rec1' , '[2] Skala 1-7' : 'rec2' , '[3] Skala 1-7' : 'rec3' ,'[4] Skala 1-7' : 'rec4' ,'[5] Skala 1-7' : 'rec5' ,'[6] Skala 1-7' : 'rec6' ,'[7] Trifft voll zu' : 'rec7'})
df_05 = df_05.drop(columns = ["[-1] keine Angabe"])
df_05.head(3)

,,syear,take_revenge,similar_problems,insult_back,rec1,rec2,rec3,rec4,rec5,rec6,rec7
pid,hid,,,,,,,,,,,
201,27,2005,[1] Trifft ueberhaupt nicht zu,[5] Skala 1-7,[1] Trifft ueberhaupt nicht zu,0,0,0,0,1,0,0
203,60313,2005,[2] Skala 1-7,[3] Skala 1-7,[2] Skala 1-7,0,0,1,0,0,0,0
602,60,2005,[5] Skala 1-7,[4] Skala 1-7,[3] Skala 1-7,0,0,0,1,0,0,0


In [263]:
# read in 2006 data
# personal = personal advancement
# still includes all unfair treat
data06 = pd.read_stata(file_paths[1], columns=["pid", "hid", "syear", 'wp43b01', 'wp43b02', 'wp43b03', 'wp43b04', 'wp43b05', 'wp43b06', 'wp43b07','wp43b08' ]).set_index(['pid', 'hid'])
df_06 = data06.rename(columns={ 'wp43b01': 'recog_sup', 'wp43b02': 'felt_recog_sup',"wp43b03": "recog_effort",  'wp43b04': 'felt_recog_effort', "wp43b05": "recog_personal", "wp43b06" :"felt_recog_personal" ,"wp43b07": "recog_pay",'wp43b08': 'felt_recog_pay'})

In [264]:
#read in 2007 data
# here left out 'xp8601' for school degree since we have it in another module also 'xp0102' : 'work_satisfaction' for the beginning
#for outcome and all controls
data3= pd.read_stata(file_paths[2], columns=["pid", "hid", "syear", 'xp13101' , 'xp13102', 'xp2701', 'xp7302','xp7202','xp28', 'xp3001' ,  'xp5701' , 'xp0102']).set_index(['pid', 'hid'])
df_07 = data3.rename(columns= {'xp13101':'gender','xp13102': 'year_birth' ,'xp2701': 'turnover_intention' , 'xp7302': 'wage_lastmonth','xp7202': 'overtime','xp28': 'new_job', 'xp3001': 'reason_new_job',  'xp5701' : 'commute_distance' , "xp0102" : "work_satisfaction"})

In [265]:
# read in 2007 data from work module

# adapt path and merge
hours07 = pd.read_stata(file_paths_2[2], columns=["pid","hid", "syear", "xvebzeit", "xpsbil", "betr07", "xerwzeit", "xbilzeit"]).set_index(['pid', 'hid'])
work07 = hours07.rename(columns={'xvebzeit': 'working_hours', "xpsbil": "school_degree", "betr07": "firmsize", "xerwzeit": "tenure" , "xbilzeit" : "years_educ"})

 ### 2. Define Functions and mappings for cleaning data

In [266]:
# mapping for reciprocity questions: same scale for all
reciprocity_questions_mapping = {
    '[1] Trifft ueberhaupt nicht zu': 1,
    '[2] Skala 1-7': 2,
    '[3] Skala 1-7': 3,
    '[4] Skala 1-7': 4,
    '[5] Skala 1-7': 5,
    '[6] Skala 1-7': 6,
    '[7] Trifft voll zu': 7,
    '[-1] keine Angabe': -1,
}
## mapping for recognition questions: binary -> binary -> unfair treatment: No -> later 1 fair treatment: Yes -> 0
recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Ja': 2,
    '[2] Nein': 1,
}
# felt recog mapping
felt_recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Gar nicht': 1,
    '[2] Maessig': 2,
    '[3] Stark': 3,
    '[4] Sehr stark': 4,
}
# mapping for firmsize -> we need to recode this in a sensible way: jumps are the same: first change: selbstständig to 0
firmsize_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Unter  5': 1,
    '[2] 5 bis 10': 2,
    '[3] 11 bis unter 20': 3,
    '[4] bis 90: unter 20': 4,
    '[5] 91-04: 5 bis unter 20': 5,
    '[6] 20 bis unter 100': 6,
    '[7] 100 bis unter 200': 7,
    '[8] bis 98: 20 bis unter 200': 8,
    '[9] 200 bis unter 2000': 9,
    '[10] 2000 und mehr': 10,
    '[11] Selbstaendig-ohne Mitarb.': 0,
}
# mapping new job into binary variable
new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1, 
    '[1] Ja': 1, 
    '[2] Nein': 2,
    '[3] Ja, nach Datenpruefung': 1,
}
# mapping for turnover intention: split up into binary with roughly equal value counts for simplicity: might change that later to categories
satisfaction_mapping = {
    '[0] 0 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[1] 1 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[2] 2 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[3] 3 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[4] 4 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[5] 5 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[6] 6 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[7] 7 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[8] 8 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[9] 9 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[10] 10 Zufrieden: Skala 0-Niedrig bis 10-Hoc': 1,
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
}
turnover_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 2,
    '[10] 10% wahrscheinlich': 1,
    '[20] 20% wahrscheinlich': 1,
    '[30] 30% wahrscheinlich': 1,
    '[40] 40% wahrscheinlich': 1, 
    '[50] 50% wahrscheinlich': 1,
    '[60] 60% wahrscheinlich': 1,
    '[70] 70% wahrscheinlich': 1,
    '[80] 80% wahrscheinlich': 1,
    '[90] 90% wahrscheinlich': 1,
    '[100] 100% wahrscheinlich': 1,
}
# mapping for turnover intention robustness check: Cardinal
turnover_mapping_cardinal = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 0,
    '[10] 10% wahrscheinlich': 10,
    '[20] 20% wahrscheinlich': 20,
    '[30] 30% wahrscheinlich': 30,
    '[40] 40% wahrscheinlich': 40, 
    '[50] 50% wahrscheinlich': 50,
    '[60] 60% wahrscheinlich': 60,
    '[70] 70% wahrscheinlich': 70,
    '[80] 80% wahrscheinlich': 80,
    '[90] 90% wahrscheinlich': 90,
    '[100] 100% wahrscheinlich': 100,
}
# mapping for new job to easier remove negatives
reason_new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1, 
    '[1] Erstmals erwerbstaetig': 1,
    '[2] Wieder erwerbstaetig': 2,
    '[3] Stelle bei neuen Arbeitgeber': 3,  
    '[4] Uerbnommen von Betrieb': 4,
    '[5] Stellenwechsel im Betrieb': 5, 
    '[6] Selbstaendig geworden': 6,
}
# mapping for school degree: to easier remove negatives
school_degree_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[1] Hauptschulabschluss': 1,
    '[2] Realschulabschluss': 2,
    '[3] Fachhochschulreife': 3,
    '[4] Abitur': 4,
    '[5] Anderer Abschluss': 5,
    '[6] Ohne Abschluss verlassen': 6,
    '[7] Noch kein Abschluss': 7,
    '[8] Keine Schule besucht': 8,
}
# reversed mapping to redo changes
reversed_mapping_reason = {v: k for k, v in reason_new_job_mapping.items()}
reversed_mapping_schoold = {v: k for k, v in school_degree_mapping.items()}
# mapping for binary reciprocity measure
def binary_reciprocity(x):
    if x >= 2:
        return 1 # High Group
    else:
        return 0 # Low Group

## function for recoding values and dropping missing

def recode_categoricals(inputdf,rc_cardinal = 0):
    """
        merges data from different years, applies recoding to categoricals and constructs additional variables.

        Input:
            - inputdf : merged dataframe
            - rc_cardinal: optional argument: if == 1 turnover intentions will be coded as cardinal instead of binary 
            - rc_rec_binary: optional argument: if == 1 neg. reciprocity will be split into 2 groups: high and low
        Output:
            - df : cleaned Dataframe

    """
    merged = inputdf  
    
    # recode Gender variable
    merged['gender'].replace('[2] Weiblich', 2,inplace=True)
    merged['gender'].replace('[1] Maennlich', 1,inplace=True)
    # recode reciprocity variables
    merged[["similar_problems","take_revenge","insult_back"]] = merged[["similar_problems","take_revenge","insult_back"]].apply(lambda x: x.map(reciprocity_questions_mapping))
    # recode recognition variables
    merged[["recog_sup","recog_effort","recog_personal","recog_pay"]] = merged[["recog_sup","recog_effort","recog_personal","recog_pay"]].apply(lambda x: x.map(recog_mapping))
    # recode felt recognition variables
    merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]] = merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]].apply(lambda x: x.map(felt_recog_mapping))
    # recode firm size
    merged['firmsize'] = merged['firmsize'].map(firmsize_mapping)
    # recode new job reason variable
    merged['reason_new_job'] = merged['reason_new_job'].map(reason_new_job_mapping)
    # recode job change variable
    merged['new_job']= merged['new_job'].map(new_job_mapping)
    # recode job satisfaction
    merged['work_satisfaction']= merged['work_satisfaction'].map(satisfaction_mapping)
    # recode turnover intention variable
    if rc_cardinal == 1:
        merged['turnover_intention'] = merged['turnover_intention'].map(turnover_mapping_cardinal)
    else:
        merged['turnover_intention'] = merged['turnover_intention'].map(turnover_mapping)
    # recode school degree
    merged['school_degree'] = merged['school_degree'].map(school_degree_mapping)
    
    output = merged
    return output

# Merge dataframes: a bit tough to read as its nested, merges 4 dataframes: 2005,2006,2007 and 2007gen

def merge_and_clean(df_05,df_06,df_07,work07,rc_cardinal = 0,rc_rec_binary=0):
    """
    merges data from different years, applies recoding to categoricals and constructs additional variables.

    Input:
        - df_05 : Pd.Dataframe contains reciprocity measures
        - df_06 : Pd.Dataframe contains unfair treatment measures
        - df_07 : Pd.Dataframe contains outcome and controls
        - work07 : Pd.Dataframe contains additional controls

    Output:
        - df : cleaned Dataframe

    """
    allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
    recoded = recode_categoricals(allmerged_df,rc_cardinal).astype('int')

    # replaces negative values with n.a.n 
    recoded = recoded.mask(recoded < 0, np.nan) 
    # construct avg reciprocity measure
    recoded['avg_rec'] = recoded[['take_revenge', 'similar_problems', 'insult_back']].mean(axis=1)
    # For robustness check: option to construct binary reciprocity measure.
    if rc_rec_binary == 1:
        recoded['binary_rec'] = recoded['avg_rec'].apply(binary_reciprocity)
    
    # construct age, potential experience and age^2
    recoded['age'] = 2007 - recoded['year_birth']
    recoded["potential_experience"] = pow((recoded["age"] - 18), 2)
    recoded["age_squared"] = (recoded["age"] ** 2) / 100
    # recode categoricals back to make it better readable
    recoded["reason_new_job"] = recoded["reason_new_job"].map(reversed_mapping_reason)
    recoded["school_degree"] = recoded["school_degree"].map(reversed_mapping_schoold)

    # transform binary variables with 1 and 2 into 1 and 0
    columns_to_transform = ["recog_sup","recog_effort", "recog_pay", "recog_personal" ,"gender", "turnover_intention", "new_job"]

    # Iterate over the columns and replace the values 2 with 0 
    for col in columns_to_transform:
        recoded[col] = recoded[col].replace({2: 0})

    # save df somewhere so its not muted when repeatedly executing this cell: Can later transform that into functions
    df = recoded
    
    return df

# Add Mincer Wage Regression and adds its residuals to the Dataframe

def add_mincer_residuals(cleaneddata):
    
    df_cleaned = cleaneddata
    # specify which columns to drop from our dataframe
    df_mincer = cleaneddata.drop(columns=['syear_x', 'similar_problems', 'take_revenge', 'insult_back','syear_y', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'syear_y', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup', 'age', 'commute_distance', 'recog_effort', 'working_hours', 'turnover_intention', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction'])
    # Convert 'gender' and 'sector' columns to categorical data type
    for col in ['gender']:
        df_mincer[col] = df_mincer[col].astype('category')
    df_mincer = df_mincer.dropna()
    # Define the dependent variable
    y = df_mincer['wage_lastmonth']
    # Define the independent variables
    X = df_mincer[['gender', 'firmsize', 'tenure', 'years_educ', 'potential_experience', 'age_squared']]

    # Add a constant term to the independent variables
    X = sm.add_constant(X)

    # Fit the Mincer wage regression model
    mincer_model = sm.OLS(y, X).fit()
    
    # Create a new column in the dataframe with the same name as the residuals array/ delete relative wage entries
    df_cleaned['mincer_residuals'] = None
    # Match the rows of the dataframe with the values in the residuals array using the index
    df_cleaned.loc[df_cleaned.index, 'mincer_residuals'] = mincer_model.resid

    return df_cleaned

### Inlcude People who switched their jobs in 2006-2007 with 2006 controls

In [267]:
def include_jobchangers(data07,onlynewemployer=False,worksatisfaction=False,rc_cardinal = False):
    """
    Input: 
            df: finished 2007 dataframe
            onlynewemployer: optional argument: If True only includes those who switched jobs to a new employer
            worksatisfaction: if True drops observations where job satisfaction between 2006 and 2007 changed to check for our Assumption
    
    """
    df = data07
    # read in cleaned 2006 dataset:
    #  change path here : Maxie: '/Users/maxieschulze/Documents/Dokumente - MacBook Pro von Maxie/5. Semester/Research Module/ResearchModule/src/data_management/2006jobchange.csv'
    #                     Max 'C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/2006jobchange.csv'  
    observations_2006 = pd.read_csv('C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/2006jobchange.csv')
    # some initial datamanagement
    observations_2006.reset_index(inplace=True)
    observations_2006.set_index(["pid","hid"], inplace=True)
    observations_2006.drop(columns=observations_2006.filter(regex='^syear').columns, inplace=True)
    observations_2006.drop("index",axis=1 ,inplace =True)
    # If cardinal turnover intentions are coded to 100
    if rc_cardinal == True:
        observations_2006["turnover_intention"] = 100
    # drop s year columns from both dataframes
    df.drop(columns=df.filter(regex='^syear').columns, inplace=True)
    
    # optional worksatisfaction robustness check
    if worksatisfaction == True:
        satisfaction = pd.DataFrame(df['work_satisfaction']).join(pd.DataFrame(observations_2006['work_satisfaction']), on = ["pid", "hid"], lsuffix = "_07" , rsuffix ="_06")
        ID_keep_satis = satisfaction[satisfaction["work_satisfaction_07"] == satisfaction["work_satisfaction_06"]].index
        df.drop(df[~df.index.isin(ID_keep_satis)].index, inplace=True)
    
    # create list of IDs of people who switched to a new employer in the last year in 2007
    if onlynewemployer == True:
        IDs_tokeep = df[(df["new_job"] == 1) & (df["reason_new_job"] == '[3] Stelle bei neuen Arbeitgeber')].index
    # if True drops people whose work satisfaction changed between 2006 and 2007
    else:
        IDs_tokeep = df[(df["new_job"] == 1)].index
    
    # drop all who changed their job in 2007 dataframe and replace 2007 controls with 2006 controls
    df.drop(df[df["new_job"] == 1].index, inplace = True)
    observations_2006.drop(observations_2006[~observations_2006.index.isin(IDs_tokeep)].index, inplace=True)
    #concat both dataframes
    dfconcat = pd.concat([df,observations_2006])
    
    return dfconcat

### 3. Analysis
#### 1.Probit Regression

- Probit Estimation: TurnoverIntention_{2005} = Constant + Neg-Rec + Unfair + Rec X Unfair + Controls + Error

- As the measure for unfair treatment in the first regression i first used recog_effort -> decide for one later
    - "When I consider all my accomplishments and efforts, the recognition of I've received seems about right to me"

In [ ]:
df_OLS = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)))
# specify columns which we need for regression by dropping everything else
df_OLS = df_OLS.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction'])
#  included 'recog_effort'again , excluded overtime due to sample size and wage last month
# add interaction term
df_OLS["recXrecog_effort"] = df_OLS["recog_effort"] * df_OLS["avg_rec"]
# drop missing data in regression dataframe
df_OLS.dropna(inplace=True)

In [ ]:
from statsmodels.discrete.discrete_model import Probit
# Define Outcome variable
Y = df_OLS["turnover_intention"]
# define X matrix
X = df_OLS.drop(columns=["turnover_intention"])
# add constant 
X = sm.add_constant(X)
# build model
model = Probit(Y, X.astype(float))
# estimate model
probit_model = model.fit()
# print summary table
probit_model.summary()

#### 2. OLS
Now, instead of using a probit model, we use OLS.  linear probability model

In [ ]:
from statsmodels.regression.linear_model import OLS
model = OLS(Y, X.astype(float))
lpm_model = model.fit(cov_type= "HC3")
lpm_model.summary()

### Robustness Checks


#### 1. Recode turnover variable into cardinal variable: Optional argument in recode_categoricals

In [274]:
# optional argument ==1
dfrc1 = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07,rc_cardinal=1)),rc_cardinal=True)
# specify columns which we need for regression by dropping everything else
dfrc_cardinal = dfrc1.drop(columns=['similar_problems', 'take_revenge', 'insult_back','felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth','rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' ,'work_satisfaction'])
#  included 'recog_effort'again , excluded overtime due to sample size and wage last month
# add interaction term
dfrc_cardinal["recXrecog_effort"] = dfrc_cardinal["recog_effort"] * dfrc_cardinal["avg_rec"]
# drop missing data in regression dataframe
dfrc_cardinal = dfrc_cardinal.dropna()

## OLS regression
from statsmodels.regression.linear_model import OLS

Y = dfrc_cardinal["turnover_intention"]
X = dfrc_cardinal.drop(columns=["turnover_intention"])
X = sm.add_constant(X)
model = OLS(Y, X.astype(float))
lpm_model = model.fit(cov_type= "HC3")

lpm_model.summary()

c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-266-d2286a43623d>:235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mincer[col] = df_mincer[col].astype('category')


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     391.6
Date:                Mon, 09 Jan 2023   Prob (F-statistic):               0.00
Time:                        16:51:29   Log-Likelihood:                -29024.
No. Observations:                5962   AIC:                         5.807e+04
Df Residuals:                    5949   BIC:                         5.816e+04
Df Model:                          12                                         
Covariance Type:                  HC3                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    8.1098      0.574     14.140      0.000       6.986       9.234
recog_effort             9.5588      2.093      4.568      0.000       5.458      13.660
working_hours           -0.2002      0.060     -3.353      0.001      -0.317      -0.083
firmsize                -0.4913      0.151     -3.256      0.001      -0.787      -0.196
tenure                  -0.9219      0.050    -18.468      0.000      -1.020      -0.824
years_educ               1.2507      0.149      8.406      0.000       0.959       1.542
gender                   3.1980      0.970      3.297      0.001       1.297       5.099
commute_distance         0.0401      0.010      4.108      0.000       0.021       0.059
avg_rec                  0.8916      0.363      2.454      0.014       0.179       1.604
age                      6.9338      0.460     15.071      0.000       6.032       7.835
potential_experience     0.2529      0.020     12.832      0.000       0.214       0.292
age_squared            -23.7772      1.695    -14.026      0.000     -27.100     -20.455
mincer_residuals        -0.0019      0.001     -3.117      0.002      -0.003      -0.001
recXrecog_effort        -0.6603      0.604     -1.093      0.274      -1.844       0.524
==============================================================================
Omnibus:                      512.954   Durbin-Watson:                   1.262
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              656.513
Skew:                           0.812   Prob(JB):                    2.75e-143
Kurtosis:                       3.062   Cond. No.                     1.18e+18
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The smallest eigenvalue is 3.73e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Does not change much. Coefficient on the interaction term becomes insignificant.

#### 2. Different Reciprocity Specifications

    1. Split into binary: High/ Low reciprocal
    2. create dummies for each category and estimate regression for those


In [273]:
# 1. optional argument:  rc_binary ==1 . Split up from when average bigger than 3
dfrc2 = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07,rc_rec_binary=1)))
# specify columns which we need for regression by dropping everything else
dfrc_binary = dfrc2.drop(columns=['similar_problems', 'take_revenge', 'insult_back','felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth','avg_rec', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction'])
# add interaction term
dfrc_binary["binaryrecXrecog_effort"] = dfrc_binary["recog_effort"] * dfrc_binary["binary_rec"]
# drop missing data in regression dataframe
dfrc_binary = dfrc_binary.dropna()
## OLS regression
from statsmodels.regression.linear_model import OLS

Y = dfrc_binary["turnover_intention"]
X = dfrc_binary.drop(columns=["turnover_intention"])
X = sm.add_constant(X)
model = OLS(Y, X.astype(float))
lpm_model = model.fit(cov_type= "HC3")
lpm_model.summary()

c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-266-d2286a43623d>:235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mincer[col] = df_mincer[col].astype('category')


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.186
Model:                            OLS   Adj. R-squared:                  0.184
Method:                 Least Squares   F-statistic:                     580.9
Date:                Mon, 09 Jan 2023   Prob (F-statistic):               0.00
Time:                        16:51:17   Log-Likelihood:                -3366.7
No. Observations:                5407   AIC:                             6759.
Df Residuals:                    5394   BIC:                             6845.
Df Model:                          12                                         
Covariance Type:                  HC3                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      0.0407      0.008      5.420      0.000       0.026       0.055
recog_effort               0.1356      0.030      4.455      0.000       0.076       0.195
working_hours             -0.0010      0.001     -1.134      0.257      -0.003       0.001
firmsize                  -0.0120      0.002     -5.347      0.000      -0.016      -0.008
tenure                    -0.0087      0.001    -11.563      0.000      -0.010      -0.007
years_educ                 0.0195      0.002      8.872      0.000       0.015       0.024
gender                     0.0507      0.014      3.515      0.000       0.022       0.079
commute_distance           0.0004      0.000      3.066      0.002       0.000       0.001
binary_rec                 0.1265      0.020      6.488      0.000       0.088       0.165
age                        0.0446      0.006      7.274      0.000       0.033       0.057
potential_experience       0.0009      0.000      3.318      0.001       0.000       0.001
age_squared               -0.1158      0.022     -5.225      0.000      -0.159      -0.072
mincer_residuals       -1.395e-05   9.21e-06     -1.516      0.130    -3.2e-05    4.09e-06
binaryrecXrecog_effort    -0.0299      0.033     -0.894      0.372      -0.095       0.036
==============================================================================
Omnibus:                    91546.626   Durbin-Watson:                   1.837
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              402.345
Skew:                          -0.045   Prob(JB):                     4.28e-88
Kurtosis:                       1.667   Cond. No.                     8.22e+17
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The smallest eigenvalue is 7.33e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

##### also does not change a lot

### 2. Creating dummies and include 7 interaction terms : Decide for one of the 3 questions for simplicity

When treating the categories as numerical: we are making assumptions about the differences between the scale items. If those distances can be considered equal at all levels, then it is reasonable to treat reciprocity as numerical. (i.e a one unit change from 1 to 2 is equivalent to a one unit change from 6 to 7)


For dummy coding we need to exclude one of the categories in the dataframe and make it the reference category: This will be the lowest level of reciprocity 1 and will be coded as zero.  so rec2 rec3 rec4 , ... and their interaction terms with unfair treatment stay in the regression.

rec2 is then interpreted as the mean of turnover intentions in the rec2 group - the mean of turnover intentions in the rec1 group (reference group) holding everything else constant

In [272]:
# 1. optional argument:  rc_binary ==1
dfrc2 = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)))
# specify columns which we need for regression by dropping everything else
dfrc_dummy = dfrc2.drop(columns=['similar_problems', 'take_revenge', 'insult_back','felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay','year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'avg_rec','rec1'])
# add interaction terms
for col in ['rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7']:
    dfrc_dummy = dfrc_dummy.assign(**{col + '_X_recog_effort': dfrc_dummy[col] * dfrc_dummy['recog_effort']})
# drop missing data in regression dataframe
dfrc_dummy = dfrc_dummy.dropna()
## OLS regression
from statsmodels.regression.linear_model import OLS

Y = dfrc_dummy["turnover_intention"]
X = dfrc_dummy.drop(columns=["turnover_intention"])
X = sm.add_constant(X)
model = OLS(Y, X.astype(float))
lpm_model = model.fit(cov_type= "HC3")
lpm_model.summary()

c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-266-d2286a43623d>:235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mincer[col] = df_mincer[col].astype('category')


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     532.8
Date:                Mon, 09 Jan 2023   Prob (F-statistic):               0.00
Time:                        16:50:52   Log-Likelihood:                -3471.6
No. Observations:                5918   AIC:                             6991.
Df Residuals:                    5894   BIC:                             7152.
Df Model:                          23                                         
Covariance Type:                  HC3                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0300      0.007      4.528      0.000       0.017       0.043
rec2                     0.1488      0.021      7.129      0.000       0.108       0.190
rec3                     0.1379      0.023      6.092      0.000       0.094       0.182
rec4                     0.1315      0.024      5.574      0.000       0.085       0.178
rec5                     0.1457      0.028      5.172      0.000       0.091       0.201
rec6                     0.1809      0.040      4.467      0.000       0.102       0.260
rec7                     0.0610      0.042      1.463      0.144      -0.021       0.143
recog_effort             0.1160      0.026      4.501      0.000       0.065       0.167
working_hours           -0.0016      0.001     -2.074      0.038      -0.003   -8.93e-05
firmsize                -0.0107      0.002     -5.258      0.000      -0.015      -0.007
tenure                  -0.0101      0.001    -14.094      0.000      -0.011      -0.009
years_educ               0.0207      0.002     10.009      0.000       0.017       0.025
gender                   0.0519      0.013      3.847      0.000       0.025       0.078
commute_distance         0.0004    9.4e-05      3.894      0.000       0.000       0.001
work_satisfaction        0.1888      0.013     14.569      0.000       0.163       0.214
age                      0.0316      0.006      5.718      0.000       0.021       0.042
potential_experience     0.0006      0.000      2.592      0.010       0.000       0.001
age_squared             -0.0858      0.020     -4.397      0.000      -0.124      -0.048
mincer_residuals       -7.9e-06    8.7e-06     -0.908      0.364   -2.49e-05    9.15e-06
rec2_X_recog_effort     -0.0578      0.035     -1.671      0.095      -0.126       0.010
rec3_X_recog_effort     -0.0143      0.036     -0.396      0.692      -0.085       0.056
rec4_X_recog_effort     -0.0888      0.039     -2.303      0.021      -0.164      -0.013
rec5_X_recog_effort     -0.0688      0.047     -1.467      0.142      -0.161       0.023
rec6_X_recog_effort     -0.1051      0.064     -1.642      0.101      -0.231       0.020
rec7_X_recog_effort     -0.0332      0.065     -0.511      0.609      -0.161       0.094
==============================================================================
Omnibus:                     2943.247   Durbin-Watson:                   1.769
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              334.242
Skew:                          -0.141   Prob(JB):                     2.63e-73
Kurtosis:                       1.870   Cond. No.                     6.95e+17
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The smallest eigenvalue is 1.06e-26. This might i

##### Robustness Check : Drop people whose job satisfaction level changed

In [271]:
dfworksatisfation = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)),worksatisfaction = True)
# specify columns which we need for regression by dropping everything else
df_rc_ws = dfworksatisfation.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction'])
#  included 'recog_effort'again , excluded overtime due to sample size and wage last month """
# add interaction term
df_rc_ws["recXrecog_effort"] = df_rc_ws["recog_effort"] * df_rc_ws["avg_rec"]
# drop missing data in regression dataframe
df_rc_ws.dropna(inplace=True)

from statsmodels.regression.linear_model import OLS

Y = df_rc_ws["turnover_intention"]
X = df_rc_ws.drop(columns=["turnover_intention"])
X = sm.add_constant(X)
model = OLS(Y, X.astype(float))
lpm_model = model.fit(cov_type= "HC3")
lpm_model.summary()

c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-266-d2286a43623d>:235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mincer[col] = df_mincer[col].astype('category')


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.192
Model:                            OLS   Adj. R-squared:                  0.190
Method:                 Least Squares   F-statistic:                     597.5
Date:                Mon, 09 Jan 2023   Prob (F-statistic):               0.00
Time:                        16:50:40   Log-Likelihood:                -2927.9
No. Observations:                4732   AIC:                             5882.
Df Residuals:                    4719   BIC:                             5966.
Df Model:                          12                                         
Covariance Type:                  HC3                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0534      0.008      6.987      0.000       0.038       0.068
recog_effort             0.1912      0.033      5.707      0.000       0.126       0.257
working_hours           -0.0013      0.001     -1.440      0.150      -0.003       0.000
firmsize                -0.0104      0.002     -4.341      0.000      -0.015      -0.006
tenure                  -0.0095      0.001    -11.565      0.000      -0.011      -0.008
years_educ               0.0209      0.002      8.946      0.000       0.016       0.026
gender                   0.0514      0.015      3.316      0.001       0.021       0.082
commute_distance         0.0004      0.000      3.339      0.001       0.000       0.001
avg_rec                  0.0237      0.006      4.072      0.000       0.012       0.035
age                      0.0546      0.006      8.637      0.000       0.042       0.067
potential_experience     0.0013      0.000      4.970      0.000       0.001       0.002
age_squared             -0.1535      0.023     -6.806      0.000      -0.198      -0.109
mincer_residuals     -1.011e-05   1.01e-05     -1.000      0.317   -2.99e-05     9.7e-06
recXrecog_effort        -0.0237      0.010     -2.432      0.015      -0.043      -0.005
==============================================================================
Omnibus:                    11899.440   Durbin-Watson:                   1.765
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              334.062
Skew:                          -0.098   Prob(JB):                     2.88e-73
Kurtosis:                       1.713   Cond. No.                     9.87e+17
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The smallest eigenvalue is 4.18e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### Robustness Check 

Avg reciprocity measure over years



In [270]:
df_OLS_avg = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)))
# specify columns which we need for regression by dropping everything else
df_OLS_avg = df_OLS_avg.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction','avg_rec'])
# Load avg reciprocity measures over the years
# Max 'C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/rec_avgyears.csv'  
avg_reciprocity = pd.read_csv('C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/rec_avgyears.csv')
avg_reciprocity.reset_index(inplace=True)
avg_reciprocity.set_index(["pid","hid"], inplace=True)
avg_reciprocity.drop("index",axis=1 ,inplace =True)

df_avg_years = pd.merge(df_OLS_avg,avg_reciprocity, on=["pid","hid"])
# add interaction term
df_avg_years["recXrecog_effort"] = df_avg_years["recog_effort"] * df_avg_years["avg_rec"]
# drop missing data in regression dataframe
df_avg_years.dropna(inplace=True)




from statsmodels.regression.linear_model import OLS

Y = df_avg_years["turnover_intention"]
X = df_avg_years.drop(columns=["turnover_intention"])
X = sm.add_constant(X)
model = OLS(Y, X.astype(float))
lpm_model = model.fit(cov_type= "HC3")
lpm_model.summary()

c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-266-d2286a43623d>:235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mincer[col] = df_mincer[col].astype('category')


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.204
Model:                            OLS   Adj. R-squared:                  0.203
Method:                 Least Squares   F-statistic:                     861.3
Date:                Mon, 09 Jan 2023   Prob (F-statistic):               0.00
Time:                        16:50:21   Log-Likelihood:                -3631.3
No. Observations:                5962   AIC:                             7289.
Df Residuals:                    5949   BIC:                             7376.
Df Model:                          12                                         
Covariance Type:                  HC3                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0531      0.007      7.866      0.000       0.040       0.066
recog_effort             0.1604      0.032      5.020      0.000       0.098       0.223
working_hours           -0.0012      0.001     -1.504      0.133      -0.003       0.000
firmsize                -0.0112      0.002     -5.348      0.000      -0.015      -0.007
tenure                  -0.0104      0.001    -14.357      0.000      -0.012      -0.009
years_educ               0.0206      0.002      9.830      0.000       0.016       0.025
gender                   0.0491      0.014      3.573      0.000       0.022       0.076
commute_distance         0.0004   9.78e-05      3.888      0.000       0.000       0.001
age                      0.0553      0.006      9.846      0.000       0.044       0.066
potential_experience     0.0013      0.000      5.486      0.000       0.001       0.002
age_squared             -0.1520      0.020     -7.648      0.000      -0.191      -0.113
mincer_residuals     -1.565e-05   8.84e-06     -1.770      0.077    -3.3e-05    1.67e-06
avg_rec                  0.0260      0.006      4.403      0.000       0.014       0.038
recXrecog_effort        -0.0173      0.009     -1.853      0.064      -0.036       0.001
==============================================================================
Omnibus:                     5582.654   Durbin-Watson:                   1.756
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              390.089
Skew:                          -0.165   Prob(JB):                     1.96e-85
Kurtosis:                       1.791   Cond. No.                     1.42e+18
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The smallest eigenvalue is 2.57e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Also does not change much